## Natural Language to SQL using Google's Gemini Pro | Python | Google AI Studio

[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

Click on the link below to open a Colab version of the notebook. You will be able to create your own version.

<a href="https://colab.research.google.com/github/bhattbhavesh91/n2sql-google-gemini/blob/main/n2sql-google-gemini-notebook.ipynb" target="_blank"><img height="40" alt="Run your own notebook in Colab" src = "https://colab.research.google.com/assets/colab-badge.svg"></a>

# Installation

In [1]:
!pip install -q google-generativeai==0.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 4.5 MB/s eta 0:00:00


# Imports

In [2]:
import google.generativeai as genai
from pathlib import Path
import sqlite3

# Version

In [3]:
genai.__version__

'0.3.1'

# Secret Key

In [4]:
from google.colab import userdata

genai.configure(api_key = userdata.get('GEMINI_KEY'))

# Configurations

In [5]:
# Set up the model
generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

# Model Instance

In [6]:
model = genai.GenerativeModel(model_name = "gemini-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

# SQL Query Executor

In [7]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [10]:
import sqlite3
import pandas as pd

# Function to load a single CSV file into an SQLite table
def load_csv_to_sqlite(csv_file, table_name, db):
    # Connect to the SQLite database
    conn = sqlite3.connect(db)
    # Load CSV into a pandas DataFrame
    df = pd.read_csv(csv_file)
    # Write the DataFrame to an SQLite table
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    # Close the connection
    conn.close()

# Function to execute and print SQL query results
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

# Load the CSV file into an SQLite table
load_csv_to_sqlite('mcdonalds.csv', 'table1', 'example.db')

# Write your SQL query
sql_query = '''
SELECT * FROM table1
'''

# Execute the query and print the results
read_sql_query(sql_query, 'example.db')


('No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', '-3', 61, 'Every three months', 'Female')
('Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', '+2', 51, 'Every three months', 'Female')
('No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', '+1', 62, 'Every three months', 'Female')
('Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', '+4', 69, 'Once a week', 'Female')
('No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', '+2', 49, 'Once a month', 'Male')
('Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', '+2', 55, 'Every three months', 'Male')
('Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', '+2', 56, 'Every three months', 'Female')
('Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'I love it!+5', 23, 'Once a week', 'Female')
('No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'I hate it!-5',

# Define Prompt

In [14]:
prompt_parts_1 = [
    """You are an expert in converting English questions to SQL code! The SQL database has the name example.db and has the following columns - yummy, convenient, spicy, fattening, greasy, fast, cheap, tasty, expensive, healthy, disgusting, Like, Age, VisitFrequency, and Gender.

    For example,
    Example 1 - How many people find McDonald's food yummy? The SQL command will be something like this:
    SELECT COUNT(*) FROM mcdonalds_data WHERE yummy = 1;

    Example 2 - How many people think McDonald's food is both cheap and tasty?
    SELECT COUNT(*) FROM mcdonalds_data WHERE cheap = 1 AND tasty = 1;

    Example 3 - What is the average age of people who visit McDonald's frequently?
    SELECT AVG(Age) FROM mcdonalds_data WHERE VisitFrequency = 'frequently';

    Do not include backticks (`) or \\n in the output."""
]


In [16]:
question = "How many people think McDonald's food is both greasy and fast?"

In [17]:
prompt_parts = [prompt_parts_1[0], question]
response = model.generate_content(prompt_parts)
response.text

'SELECT COUNT(*) FROM mcdonalds_data WHERE greasy = 1 AND fast = 1;'

In [19]:
read_sql_query("""SELECT COUNT(*) FROM table1 WHERE greasy = 1 AND fast = 1;
""",
               "example.db")

(0,)


# Combine it into Function

In [26]:
import sqlite3
import pandas as pd

# Function to load CSV into SQLite table
def load_csv_to_sqlite(csv_file, table_name, db):
    # Connect to the SQLite database
    conn = sqlite3.connect(db)
    # Load CSV into a pandas DataFrame
    df = pd.read_csv(csv_file)
    # Write the DataFrame to an SQLite table
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    # Close the connection
    conn.close()

# Load the CSV file into the 'mcdonalds_data' table
load_csv_to_sqlite('mcdonalds.csv', 'mcdonalds_data', 'example.db')


In [27]:
def generate_gemini_response(question, input_prompt):
    prompt_parts = [input_prompt, question]
    response = model.generate_content(prompt_parts)
    output = read_sql_query(response.text, "example.db")
    return output

In [28]:
generate_gemini_response("How many people like mcdonalds food but find it expensive?",
                         prompt_parts_1[0])

(0,)


In [48]:
import sqlite3
import pandas as pd

# Function to load the CSV file into the SQLite database
def load_csv_to_sqlite(csv_file, table_name, db):
    conn = sqlite3.connect(db)
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

# Function to generate SQL query, execute it, and print the results
def execute_query(question, input_prompt, db):
    # Generate the SQL query based on the question and prompt
    prompt_parts = [input_prompt, question]
    response = model.generate_content(prompt_parts)
    sql_query = response.text.strip()  # Ensure no leading/trailing whitespace

    # Print the SQL query for debugging purposes
    print(f"Generated SQL Query: {sql_query}")

    # Execute the SQL query and print the results
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    try:
        cur.execute(sql_query)
        rows = cur.fetchall()
        if rows:
            for row in rows:
                print(row)
        else:
            print("No results found.")
    except sqlite3.OperationalError as e:
        print(f"SQL Error: {e}")
    finally:
        conn.close()

# Example usage
# Ensure to replace 'mcdonalds.csv' with your actual CSV file path and set 'model' properly
load_csv_to_sqlite('mcdonalds.csv', 'mcdonalds_data', 'example.db')

# Define your prompt for generating SQL queries
prompt_parts_1 = [
    '''You are an expert in converting English questions to SQL code! The SQL database has the name `example.db` and contains a table named `mcdonalds_data` with the following columns:

- `yummy` (Yes/No)
- `convenient` (Yes/No)
- `spicy` (Yes/No)
- `fattening` (Yes/No)
- `greasy` (Yes/No)
- `fast` (Yes/No)
- `cheap` (Yes/No)
- `tasty` (Yes/No)
- `expensive` (Yes/No)
- `healthy` (Yes/No)
- `disgusting` (Yes/No)
- `Like` (numeric scale, e.g., -3)
- `Age` (integer)
- `VisitFrequency` (e.g., 'Every three months')
- `Gender` (e.g., 'Female')

For example,
Example 1 - How many people find McDonald's food tasty?
`SELECT COUNT(*) FROM mcdonalds_data WHERE tasty = 'Yes';`

Example 2 - What is the average age of people who think McDonald's food is both greasy and cheap?
`SELECT AVG(Age) FROM mcdonalds_data WHERE greasy = 'Yes' AND cheap = 'Yes';`

Example 3 - How many people who find McDonald's food both convenient and fast have a Like score greater than 0?
`SELECT COUNT(*) FROM mcdonalds_data WHERE convenient = 'Yes' AND fast = 'Yes' AND Like > 0;`

Example 4 - What percentage of people consider McDonald's food expensive but still find it tasty?
`SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM mcdonalds_data)) AS percentage FROM mcdonalds_data WHERE expensive = 'Yes' AND tasty = 'Yes';`

Example 5 - What is the most common visit frequency among people who think McDonald's food is fattening?
`SELECT VisitFrequency FROM mcdonalds_data WHERE fattening = 'Yes' GROUP BY VisitFrequency ORDER BY COUNT(*) DESC LIMIT 1;`

Example 6 - How many of the first 10 people find McDonald's food yummy?
`SELECT COUNT(*) FROM (SELECT * FROM mcdonalds_data LIMIT 10) AS limited_data WHERE yummy = 'Yes';`
Example 7 - Determine the average age of customers who like healthy food but dislike greasy food, and also analyze their visit frequency and gender distribution
`SELECT
  AVG(Age) AS average_age,
  VisitFrequency,
  Gender
FROM mcdonalds_data
WHERE
  healthy = 'Yes' AND greasy = 'No'
GROUP BY
  VisitFrequency,
  Gender;`

Example 8 - Analyze customer segments based on their food preferences (e.g., "yummy," "spicy"), and demographic and behavioral factors like age, visit frequency, and gender.`WITH PreferenceSegments AS (
    `WITH PreferenceSegments AS (
    SELECT
        Gender,
        VisitFrequency,
        Age,
        CASE
            WHEN yummy = 'Yes' AND healthy = 'Yes' THEN 'Yummy and Healthy'
            WHEN spicy = 'Yes' THEN 'Spicy Lovers'
            WHEN fast = 'Yes' THEN 'Fast Food Lovers'
            WHEN cheap = 'Yes' THEN 'Price Conscious'
            ELSE 'Other Preferences'
        END AS Food_Preference,
        CASE
            WHEN Like > 0 THEN 'Positive'
            WHEN Like < 0 THEN 'Negative'
            ELSE 'Neutral'
        END AS Like_Sentiment
    FROM
        your_table_name
    WHERE
        Gender IS NOT NULL
)
SELECT
    ps.Gender,
    ps.VisitFrequency,
    ps.Food_Preference,
    COUNT(*) AS Segment_Size,
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY ps.Gender)), 2) AS Segment_Percentage,
    RANK() OVER (PARTITION BY ps.Gender ORDER BY COUNT(*) DESC) AS Rank_Within_Gender,
    (CASE
         WHEN AVG(ps.Age) > 40 THEN 'Older Age Group'
         ELSE 'Younger Age Group'
     END) AS Age_Group,
    ps.Like_Sentiment
FROM
    PreferenceSegments ps
GROUP BY
    ps.Gender,
    ps.VisitFrequency,
    ps.Food_Preference,
    ps.Like_Sentiment
HAVING
    Segment_Size > 5  -- Exclude segments with fewer than 5 customers
ORDER BY
    ps.Gender,
    Rank_Within_Gender;`

Do not include backticks (`) or `\\n` in the output.'''
]

# Example questions
questions = [
    "Determine the average age of customers who like healthy food but dislike greasy food, and also analyze their visit frequency and gender distribution.",
    "Analyze customer segments based on their food preferences (e.g., 'yummy,' 'spicy'), and demographic and behavioral factors like age, visit frequency, and gender." ,
    "perform segmentation analysis"
    ]

# Execute and print results for each random question
for question in questions:
    print(f"\nQuestion: {question}")
    execute_query(question, prompt_parts_1[0], 'example.db')



Question: Determine the average age of customers who like healthy food but dislike greasy food, and also analyze their visit frequency and gender distribution.
Generated SQL Query: SELECT
  AVG(Age) AS average_age,
  VisitFrequency,
  Gender
FROM mcdonalds_data
WHERE
  healthy = 'Yes' AND greasy = 'No'
GROUP BY
  VisitFrequency,
  Gender;
(52.61290322580645, 'Every three months', 'Female')
(53.92307692307692, 'Every three months', 'Male')
(40.5, 'More than once a week', 'Female')
(45.375, 'More than once a week', 'Male')
(55.0, 'Never', 'Female')
(47.191489361702125, 'Once a month', 'Female')
(46.96, 'Once a month', 'Male')
(43.55, 'Once a week', 'Female')
(44.083333333333336, 'Once a week', 'Male')
(53.333333333333336, 'Once a year', 'Female')
(59.77777777777778, 'Once a year', 'Male')

Question: Analyze customer segments based on their food preferences (e.g., 'yummy,' 'spicy'), and demographic and behavioral factors like age, visit frequency, and gender.
Generated SQL Query: WITH Pr